# 農場IoTデータを用いて回帰モデル作成
## 収穫量の予測モデル構築

In [2]:
import findspark
findspark.init('/home/yoshi-1/spark-3.1.1-bin-hadoop2.7')

from pyspark.sql import SparkSession
from pyspark.sql.types import *

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

## sparksessionインスタンス作成、スキーマ定義、データ参照、DataFrame生成

In [3]:
# sparksessionインスタンス
ss = SparkSession\
        .builder\
        .appName("Regression")\
        .enableHiveSupport()\
        .getOrCreate()

In [4]:
# スキーマ定義
struct = StructType([
        StructField('Year', StringType(), False),
        StructField('FarmID', DoubleType(), False),
        StructField('MeanHighestTemperature', DoubleType(), False),
        StructField('MeanMinimumTemperature', DoubleType(), False),
        StructField('MeanWhc', DoubleType(), False),
        StructField('MeanDaylightHours', DoubleType(), False),
        StructField('MeanDayOfSoilAcidityRange', DoubleType(), False),
        StructField('TotalYield', DoubleType(), False),
        StructField('Area', DoubleType(), False),
        StructField('YieldPerA', DoubleType(), False),
        StructField('label', DoubleType(), False)
        ])

In [6]:
# 6月データでDataFrame作成
df6 = ss.read.csv('./batchAnalysticsData_train_6.csv',
                 header=True, encoding='UTF-8', schema=struct)
df6.show(10)

+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|Year|FarmID|MeanHighestTemperature|MeanMinimumTemperature|MeanWhc|MeanDaylightHours|MeanDayOfSoilAcidityRange|TotalYield|Area|YieldPerA|label|
+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|2007|   1.0|                 11.08|                  2.94|   14.5|           166.99|                     21.0|1423222.21| 4.5|  3162.72|  0.0|
|2007|   2.0|                 11.18|                  2.94|   14.7|           169.09|                     21.0|1457585.51| 5.0|  2915.17|  0.0|
|2007|   3.0|                 11.98|                  3.04|   13.9|           171.49|                     21.0|1150258.61| 3.0|   3834.2|  1.0|
|2007|   4.0|                 10.58|                  3.34|   16.4|           166.39|                     21.0|2327859.58| 6.0|  3879.77

In [7]:
# データセットの分割
df6TrainData, df6TestData = df6.randomSplit([0.7, 0.3], 50)

## 線形回帰モデル作成

In [8]:
# 特徴量候補
# 1. 畑×土壌酸度範囲内日数×平均最高気温
# 2. 畑×土壌酸度範囲内日数×平均最低気温
# 3. 畑×土壌酸度範囲内日数×平均含水気温
# 4. 畑×土壌酸度範囲内日数×日照合計時間

# 上記組み合わせのVectorAssemberを定義

# VectorAssembler用リスト
assemblerForLR = []

# 各VectorAssembler作成
assemblerForLR.append(
        VectorAssembler(inputCols=[
            "FarmID",
            "MeanDayOfSoilAcidityRange",
            "MeanHighestTemperature"],
            outputCol="features")
        )
assemblerForLR.append(
        VectorAssembler(inputCols=[
            "FarmID",
            "MeanDayOfSoilAcidityRange",
            "MeanMinimumTemperature"],
            outputCol="features")
        )
assemblerForLR.append(
        VectorAssembler(inputCols=[
            "FarmID",
            "MeanDayOfSoilAcidityRange",
            "MeanWhc"],
            outputCol="features")
        )
assemblerForLR.append(
        VectorAssembler(inputCols=[
            "FarmID",
            "MeanDayOfSoilAcidityRange",
            "MeanDaylightHours"],
            outputCol="features")
        )

In [9]:
# ML Pipelineの定義

# 標準化
scalerForLR = StandardScaler(inputCol="features", outputCol="standardedFeature",
                            withStd=True, withMean=True)

In [10]:
# 線型回帰インスタンス
regressionByLR = LinearRegression()\
                    .setLabelCol("YieldPerA")\
                    .setFeaturesCol("standardedFeature")\
                    .setStandardization(True)

In [11]:
# ML Pipeline
pipelineForLR = []
for assembler in assemblerForLR:
    pipelineForLR.append(
        Pipeline(
            stages=[
                assembler,
                scalerForLR,
                regressionByLR
            ]
        )
    )

In [12]:
# グリッドサーチ定義
paramGridForLR = ParamGridBuilder()\
                    .addGrid(regressionByLR.regParam, [0.001, 0.01, 0.1, 1.0, 10.0, 100.0])\
                    .addGrid(regressionByLR.maxIter, [10, 100, 1000])\
                    .addGrid(regressionByLR.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])\
                    .build()

In [13]:
# 評価用インスタンス
evaluatorForLR = RegressionEvaluator().setLabelCol("YieldPerA").setMetricName("rmse")

In [14]:
# クロスバリデーション定義
crossValidatorForLR = []
for pipeline in pipelineForLR:
    crossValidatorForLR.append(
        CrossValidator()\
            .setEstimator(pipeline)\
            .setEvaluator(evaluatorForLR)\
            .setEstimatorParamMaps(paramGridForLR)\
            .setNumFolds(5)
    )

In [16]:
# モデルを作成し、訓練データで予測

# 訓練データ
print("-- df6TrainData --")
df6TrainData.show(10)

-- df6TrainData --
+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|Year|FarmID|MeanHighestTemperature|MeanMinimumTemperature|MeanWhc|MeanDaylightHours|MeanDayOfSoilAcidityRange|TotalYield|Area|YieldPerA|label|
+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|2007|   1.0|                 11.08|                  2.94|   14.5|           166.99|                     21.0|1423222.21| 4.5|  3162.72|  0.0|
|2007|   2.0|                 11.18|                  2.94|   14.7|           169.09|                     21.0|1457585.51| 5.0|  2915.17|  0.0|
|2007|   3.0|                 11.98|                  3.04|   13.9|           171.49|                     21.0|1150258.61| 3.0|   3834.2|  1.0|
|2007|   4.0|                 10.58|                  3.34|   16.4|           166.39|                     21.0|232785

In [17]:
%%time
# モデル作成
modelByLR = []
for crossValidator in crossValidatorForLR:
    modelByLR.append(crossValidator.fit(df6TrainData))

CPU times: user 55.8 s, sys: 34.7 s, total: 1min 30s
Wall time: 6min 53s


In [18]:
# 訓練データで予測を行い、RMSEを出力
print("-- RMSE-TrainData(Linear Regression) --")
for i, model in enumerate(modelByLR):
    prediction = model.transform(df6TrainData)
    rmse = evaluatorForLR.evaluate(prediction)
    print(i, rmse)

-- RMSE-TrainData(Linear Regression) --
0 471.9291822630905
1 487.2582855969672
2 480.9451639009866
3 383.2803016030876


In [19]:
# 上記より、3. 畑×土壌酸度範囲内日数×日照合計時間 の組み合わせが最適

## 回帰木モデルの作成

In [20]:
from pyspark.ml.regression import DecisionTreeRegressor

In [23]:
# ML Pipelineの定義
assemblerForDR = VectorAssembler(
                    inputCols=[
                        "FarmID",
                        "MeanHighestTemperature",
                        "MeanMinimumTemperature",
                        "MeanWhc",
                        "MeanDaylightHours",
                        "MeanDayOfSoilAcidityRange"
                    ],
                    outputCol="features")

In [24]:
# 回帰木インスタンス
regressionByDR = DecisionTreeRegressor()\
                    .setLabelCol("YieldPerA")\
                    .setFeaturesCol("features")

In [25]:
# pipeline作成
pipelineForDR = Pipeline(
                    stages=[
                        assemblerForDR,
                        regressionByDR
                    ])

In [26]:
# グリッドサーチ定義
paramGridForDR = ParamGridBuilder()\
                    .addGrid(regressionByDR.maxBins, [10, 20, 30, 40, 50])\
                    .addGrid(regressionByDR.maxDepth, [2, 3, 4])\
                    .build()

In [27]:
# 評価用インスタンス
evaluatorForDR = RegressionEvaluator()\
                    .setLabelCol("YieldPerA")\
                    .setMetricName("rmse")

In [28]:
# クロスバリデーション定義
crossValidatorForDR = CrossValidator()\
                        .setEstimator(pipelineForDR)\
                        .setEvaluator(evaluatorForDR)\
                        .setEstimatorParamMaps(paramGridForDR)\
                        .setNumFolds(5)

In [29]:
# モデルを作成し、訓練データで予測

modelByDR = crossValidatorForDR.fit(df6TrainData)

predictionTrainDataByDR = modelByDR.transform(df6TrainData)

rmseTrainDataByDR = evaluatorForDR.evaluate(predictionTrainDataByDR)

print("-- RMSE-TrainData(DecisionTree Regression) --")
print(rmseTrainDataByDR)

-- RMSE-TrainData(DecisionTree Regression) --
363.7301426815155


## テストデータでrmse求め、モデルを選択

In [30]:
predictionTestDataByLR = modelByLR[3].transform(df6TestData)
rmseTestDataByLR = evaluatorForLR.evaluate(predictionTestDataByLR)
print("-- RMSE-TestData(LinearRegression Regression) --")
print(rmseTestDataByLR)

-- RMSE-TestData(LinearRegression Regression) --
411.9385137962273


In [31]:
predictionTestDataByDR = modelByDR.transform(df6TestData)
rmseTestDataByDR = evaluatorForDR.evaluate(predictionTestDataByDR)
print("-- RMSE-TestData(DesicionTree Regression) --")
print(rmseTestDataByDR)

-- RMSE-TestData(DesicionTree Regression) --
526.2378016592534


In [32]:
# 上記より、線型回帰モデルを選択する

In [34]:
# 未知データを、線型回帰モデルで予測

# 未知データ読み込み
df6Predict = ss.read.csv('./batchAnalysticsData_predict_6.csv',
                        header=True, encoding='UTF-8', schema=struct)
df6Predict.show(5)

+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|Year|FarmID|MeanHighestTemperature|MeanMinimumTemperature|MeanWhc|MeanDaylightHours|MeanDayOfSoilAcidityRange|TotalYield|Area|YieldPerA|label|
+----+------+----------------------+----------------------+-------+-----------------+-------------------------+----------+----+---------+-----+
|2017|   1.0|                 11.18|                  2.54|   15.8|           167.49|                     23.0|      null|null|     null| null|
|2017|   2.0|                 10.38|                  2.24|   15.3|           173.39|                     23.0|      null|null|     null| null|
|2017|   3.0|                 10.88|                  3.24|   15.6|           170.99|                     22.0|      null|null|     null| null|
|2017|   4.0|                 11.58|                  2.44|   13.4|           171.79|                     21.0|      null|null|     null

In [35]:
# 予測
predictionFutureDataByLR = modelByLR[3].transform(df6Predict)

In [37]:
# 実測値と予測結果表示

print("-- LastYearFactData --")
df6.filter(df6["Year"] == "2016").select("Year", "FarmID", "YieldPerA").show(10)

print("\n-- RMSE-FutureData(LinerRegression) --")
predictionFutureDataByLR.select("FarmID", "Prediction").show(10)

-- LastYearFactData --
+----+------+---------+
|Year|FarmID|YieldPerA|
+----+------+---------+
|2016|   1.0|  3550.63|
|2016|   2.0|   3410.7|
|2016|   3.0|  4156.91|
|2016|   4.0|  3728.93|
|2016|   5.0|  3386.44|
+----+------+---------+


-- RMSE-FutureData(LinerRegression) --
+------+------------------+
|FarmID|        Prediction|
+------+------------------+
|   1.0| 4147.876437869045|
|   2.0| 4695.928014020988|
|   3.0| 4129.276971432694|
|   4.0| 3849.910979806184|
|   5.0|2895.7476037246724|
+------+------------------+

